<a href="https://colab.research.google.com/github/MuhammadIrzam447/visionCodes/blob/master/Train_ViT_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !gdown https://drive.google.com/uc?id=14u9sgibcvl5-2oThkjYWlgYPvgjp0RhV

Downloading...
From: https://drive.google.com/uc?id=14u9sgibcvl5-2oThkjYWlgYPvgjp0RhV
To: /content/imgs-train+test.zip
 18% 246M/1.36G [00:01<00:05, 199MB/s]Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 151, in main
    filename = download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 275, in download
    f.write(chunk)
KeyboardInterrupt
 19% 259M/1.36G [00:02<00:08, 124MB/s]
^C


In [2]:
# !unzip "/content/imgs-train+test.zip"

Archive:  /content/imgs-train+test.zip
replace img-train+test/test/0/01285.jpg_4.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!pip install transformers

In [4]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from PIL import Image

In [5]:
# Define the path to your training and validation data
train_data_root = "/content/img-train+test/train"
val_data_root = "/content/img-train+test/test"

In [6]:
# from transformers import ViTImageProcessor

# processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
# image_mean, image_std = processor.image_mean, processor.image_std
# size = processor.size["height"]

# # Define transformations for the input images
# transform = transforms.Compose([
#     transforms.Resize(size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=image_mean, std=image_std)
# ])


In [7]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_folder = ImageFolder(root_dir, transform=None)

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        image_path, label = self.image_folder.imgs[idx]
        image = self.load_image(image_path)
        return image, label

    def load_image(self, path):
        image = Image.open(path).convert("RGB")
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        image = transform(image)
        return image

In [8]:
# Load the dataset using ImageFolder and apply transformations
train_dataset = ImageDataset(train_data_root)
val_dataset = ImageDataset(val_data_root)

In [9]:
# Create label2id and id2label dictionaries based on the class names in the dataset
label2id = {class_name: idx for class_name, idx in train_dataset.image_folder.class_to_idx.items()}
id2label = {idx: class_name for class_name, idx in label2id.items()}

In [28]:
def custom_collate(batch):
    # print("item1: ", batch[0])
    # print("item2: ", batch[1])

    images = [feature_extractor(images=item[0], return_tensors="pt") for item in batch]
    images = [item['pixel_values'] for item in images]
    labels = [torch.tensor(item[1]) for item in batch]

    images, labels = zip(*batch)
    images = torch.stack(images, dim=0)
    labels = torch.tensor(labels)

    return images, labels

In [29]:
# Initialize the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# Define batch size and number of workers (adjust based on your system's resources)
batch_size = 4
num_workers = 1

# Create DataLoader for the dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn= custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn= custom_collate)

In [12]:
num_classes = len(train_dataset.image_folder.classes)
print(num_classes)

2


In [13]:
vit = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", id2label=id2label, label2id=label2id, num_labels=num_classes)
vit.classifier = nn.Linear(vit.config.hidden_size, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit.to(device)
print(vit)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [14]:
criterion = nn.CrossEntropyLoss()
# optimizer = AdamW(vit.parameters(), lr=1e-5)
optimizer = optim.SGD(vit.parameters(), lr=0.001, momentum=0.9)
num_epochs = 20

In [ ]:
for epoch in range(num_epochs):
    vit.train()
    train_loss = 0.0

    for images, labels in train_loader:
        # print(f"images.shape {images.shape}")
        # print(f"labels.shape {labels.shape}")
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = vit(images)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # Calculate average loss for this epoch
    train_loss /= len(train_loader.dataset)

    save_dir = "/content/Train-ViT-02/"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    model_name = str(epoch+1) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(vit.state_dict(), save_path)

    # Validation
    vit.eval()
    val_loss = 0.0
    correct = 0
    predicted_classes = []
    actual_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = vit(images)
            val_loss += criterion(outputs.logits, labels).item() * images.size(0)

            probabilities = torch.softmax(outputs.logits, dim=1)
            predicted = torch.argmax(probabilities, dim=1)

            correct += (predicted == labels).sum().item()

            predicted_classes.extend(predicted.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    # Calculate average loss and accuracy for validation set
    val_loss /= len(val_loader.dataset)
    accuracy = correct / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {train_loss:.4f} - Validation Loss: {val_loss:.4f} - Accuracy: {accuracy:.4f}")

    # Compute evaluation metrics using the predicted_classes and actual_labels lists
    accuracy = accuracy_score(actual_labels, predicted_classes)
    precision = precision_score(actual_labels, predicted_classes, average='weighted')
    recall = recall_score(actual_labels, predicted_classes, average='weighted')
    f1 = f1_score(actual_labels, predicted_classes, average='weighted')

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print(classification_report(actual_labels, predicted_classes))
    print("Confusion Matrix:")
    print(confusion_matrix(actual_labels, predicted_classes))
    print("AUROC:", roc_auc_score(actual_labels, predicted_classes))



Epoch 1/20 - Training Loss: 0.6359 - Validation Loss: 0.6308 - Accuracy: 0.6485
Accuracy: 0.6485
Precision: 0.6560787090909803
Recall: 0.6485
F1-score: 0.5616483191735859
              precision    recall  f1-score   support

           0       0.65      0.96      0.77      1250
           1       0.67      0.12      0.21       750

    accuracy                           0.65      2000
   macro avg       0.66      0.54      0.49      2000
weighted avg       0.66      0.65      0.56      2000

Confusion Matrix:
[[1205   45]
 [ 658   92]]
AUROC: 0.5433333333333333
Epoch 2/20 - Training Loss: 0.6012 - Validation Loss: 0.6660 - Accuracy: 0.6045
Accuracy: 0.6265
Precision: 0.589772159554048
Recall: 0.6265
F1-score: 0.5491536312236911
              precision    recall  f1-score   support

           0       0.64      0.93      0.76      2500
           1       0.51      0.13      0.20      1500

    accuracy                           0.63      4000
   macro avg       0.57      0.53      0.48